In [1]:
import json
import cv2
import os
import matplotlib.pyplot as plt
import shutil

In [2]:
input_path = "data_fashion/instances/"
output_path = "data_fashion/instances/"

In [3]:
f = open(input_path + 'instances_train.json')
data = json.load(f)
f.close()

In [4]:
data.keys()

dict_keys(['annotations', 'images', 'info', 'licenses', 'categories', 'attributes'])

In [5]:
def get_img_ann(image_id):
    img_ann = []
    isFound = False
    for ann in data['annotations']:
        if ann['image_id'] == image_id:
            img_ann.append(ann)
            isFound = True
    if isFound:
        return img_ann
    else:
        return None

In [6]:
file_names = []
for img in data['images']:
    file_names.append(str(img['id'])+".jpg")

In [7]:
file_info = {}
for filename in file_names:
    img_id = int(filename[:-4])
    for img in data["images"]:
        if img["id"] == img_id:
            img_w = img['width']
            img_h = img['height']
            break
    file_info.update(
        {
            img_id: {
                "file_name": filename,
                "width": img_w,
                "height": img_h
            }
        }
    )

In [ ]:
for ann in data["annotations"]:
    file_details = file_info[ann["image_id"]]
    current_category = ann['category_id'] - 1 # As yolo format labels start from 0 
    current_bbox = ann['bbox']

    img_id_str = str(ann["image_id"])
    img_w = file_details["width"]
    img_h = file_details["height"]

    x = current_bbox[0]
    y = current_bbox[1]
    w = current_bbox[2]
    h = current_bbox[3]
    
    # Finding midpoints
    x_centre = (x + (x+w))/2
    y_centre = (y + (y+h))/2
    
    # Normalization
    x_centre = x_centre / img_w
    y_centre = y_centre / img_h
    w = w / img_w
    h = h / img_h
    
    # Limiting upto fix number of decimal places
    x_centre = format(x_centre, '.8f')
    y_centre = format(y_centre, '.8f')
    w = format(w, '.8f')
    h = format(h, '.8f')

    with open(f"{output_path}labels_new/{img_id_str}.txt", "a") as f:
        f.write(f"{current_category} {x_centre} {y_centre} {w} {h}\n")
        f.close()